In [1]:
%matplotlib widget

import pathlib
import psutil
from os import path as op

import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.simulation import simulate_evoked

import nilearn.plotting

from ipywidgets import interact, fixed, IntSlider, interactive

data_path = pathlib.Path(mne.datasets.sample.data_path())
subjects_dir = data_path / 'subjects'
subject = 'sample'
meg_path = data_path / 'MEG' / subject

data_path = mne.datasets.sample.data_path()
subjects_dir = op.join(data_path, 'subjects')
fname_ave = op.join(data_path, 'MEG', 'sample', 'sample_audvis-ave.fif')
fname_bem = op.join(subjects_dir, 'sample', 'bem',
                    'sample-5120-5120-5120-bem-sol.fif')
bem = mne.read_bem_solution(fname_bem, verbose=False)
fname_trans = op.join(data_path, 'MEG', 'sample',
                      'sample_audvis_raw-trans.fif')
trans = mne.read_trans(fname_trans)
trans_mri_to_head = mne.transforms.invert_transform(trans)
t1_fname = op.join(subjects_dir, subject, 'mri', 'T1.mgz')
t1_img = nilearn.image.load_img(t1_fname)

info = mne.io.read_info(fname_ave, verbose=False)
info['bads'] = []
info['projs'] = []
picks = mne.pick_types(info, meg=True, eeg=True)
info = mne.pick_info(info, picks)

In [2]:
def _get_axes():
    widths = [3, 3, 3, 3, 3]
    heights = [5, 5]

    gs_kw = dict(width_ratios=widths, height_ratios=heights)
    fig, fig_axes = plt.subplots(ncols=5, nrows=2,
                          gridspec_kw=gs_kw, figsize=(10, 3))
    gs = fig_axes[1, 2].get_gridspec()

    axes = []
    axes.append(fig.add_subplot(gs[:2, :2], projection='3d'))
    axes.append(fig.add_subplot(gs[:2, 2]))
    axes.append(fig.add_subplot(gs[:2, 3]))
    axes.append(fig.add_subplot(gs[:2, 4]))
    return axes, fig_axes

In [3]:
fwds = dict()

z_min, z_max = 60, 120
z_vals = np.arange(z_min, z_max + 1, 10)
x_dip, y_dip = 20, 20

for z in z_vals:
    print(f'Generating dipole fwd solution for depth z={z} mm.')
    ori = np.eye(3)
    dip_pos = (x_dip, y_dip, z)
    pos = np.array([dip_pos]) / 1e3
    pos = np.tile(pos, (3, 1))
    dip = mne.Dipole(times=np.arange(3), pos=pos, amplitude=3 * [10e-9], ori=ori, gof=3 * [100])
    fwd, _ = mne.make_forward_dipole(dip, fname_bem, info, fname_trans)    
    fwds[dip_pos] = fwd

del dip, fwd, pos, z

Generating dipole fwd solution for depth z=60 mm.
Positions (in meters) and orientations
3 sources
Source space          : <SourceSpaces: [<discrete, n_used=3>] head coords>
MRI -> head transform : /Users/hoechenberger/mne_data/MNE-sample-data/MEG/sample/sample_audvis_raw-trans.fif
Measurement data      : instance of Info
Conductor model   : /Users/hoechenberger/mne_data/MNE-sample-data/subjects/sample/bem/sample-5120-5120-5120-bem-sol.fif
Accurate field computations
Do computations in head coordinates
Free source orientations

Read 1 source spaces a total of 3 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999310  0.009985 -0.035787      -3.17 mm
     0.012759  0.812405  0.582954       6.86 mm
     0.034894 -0.583008  0.811716      28.88 mm
     0.000000  0.000000  0.000000       1.00

Read 306 MEG channels from info
99 coil definitions read
Coordinate transformation: MEG device -> head
     0.991420 -0.039936 -0.124467      -6.13 mm
     0.060661

     0.060661  0.984012  0.167456       0.06 mm
     0.115790 -0.173570  0.977991      64.74 mm
     0.000000  0.000000  0.000000       1.00
MEG coil definitions created in head coordinates.
Read  60 EEG channels from info
Head coordinate coil definitions created.
Source spaces are now in head coordinates.

Setting up the BEM model using /Users/hoechenberger/mne_data/MNE-sample-data/subjects/sample/bem/sample-5120-5120-5120-bem-sol.fif...

Loading surfaces...
Three-layer model surfaces loaded.

Loading the solution matrix...

Loaded linear_collocation BEM solution from /Users/hoechenberger/mne_data/MNE-sample-data/subjects/sample/bem/sample-5120-5120-5120-bem-sol.fif
Employing the head->MRI coordinate transform with the BEM model.
BEM model sample-5120-5120-5120-bem-sol.fif is now set up

Source spaces are in head coordinates.
Checking that the sources are inside the surface (will take a few...)
    Skipping interior check for 3 sources that fit inside a sphere of radius   43.6 mm
    

    Skipping solid angle check for 0 points using Qhull

Setting up compensation data...
    No compensation set. Nothing more to do.

Composing the field computation matrix...
Setting up for EEG...
Computing MEG at 3 source locations (free orientations)...
Computing EEG at 3 source locations (free orientations)...

Finished.
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]


In [6]:
def plot(phi, theta, z_pos, t1_img, info, fwds, axes, fig_axes):
    phi_rad = np.deg2rad(phi)
    theta_rad = np.deg2rad(theta)

    ox = np.sin(theta_rad) * np.cos(phi_rad)
    oy = np.sin(theta_rad) * np.sin(phi_rad)
    oz = np.cos(theta_rad)

    ori = np.array([[ox, oy, oz]])
    dip_pos = (x_dip, y_dip, z_pos)
    leadfield = fwds[dip_pos]['sol']['data']
    meeg_data = np.dot(leadfield, ori.T)  # compute forward
    evoked = mne.EvokedArray(meeg_data, info)

    # Do the actual plotting.
    [ax.clear() for ax in axes]
    for ax_num, ch_type in enumerate(['mag', 'grad', 'eeg'], start=1):
        evoked.plot_topomap(ch_type=ch_type, colorbar=False,
                            outlines='skirt',
                            nrows=1, ncols=1,
                            times=evoked.times[-1],
                            res=256, show=False,
                            axes=axes[ax_num])
        axes[ax_num].set_title(ch_type, fontweight='bold')

    ori = np.array([[ox, oy, oz]])
    pos = np.array([dip_pos]) / 1e3
    dip = mne.Dipole(times=[0], pos=pos, amplitude=[10e-9], ori=ori, gof=[100])

    dip.plot_locations(trans=trans, subject=subject, subjects_dir=subjects_dir,
                       ax=axes[0], show=False, coord_frame='head')
    axes[0].axis('off')
    axes[0].set_title(None)
    [x.axis('off') for x in fig_axes.ravel()]

if 0:
    axes, fig_axes = _get_axes()
    plot(phi=0, theta=0, z_pos=60, t1_img=t1_img, info=info, axes=axes, fwds=fwds, fig_axes=fig_axes)

In [11]:
style = {'description_width': 'initial'}

phi_slider = IntSlider(description='azimuth φ (deg)', min=0, max=360, step=1, value=0,
                       continuous_update=False, style=style)
theta_slider = IntSlider(description='polar angle θ (deg)', min=-90, max=90, step=1, value=45,
                         continuous_update=False, style=style)
z_slider = IntSlider(description='z pos (mm)', min=z_min, max=z_max, step=10,
                     value=80, continuous_update=False, orientation='vertical', style=style)


axes, fig_axes = _get_axes()
interactive_plot =  interactive(plot,
                                phi=phi_slider,
                                theta=theta_slider,
                                z_pos=z_slider,
                                t1_img=fixed(t1_img),
                                fwds=fixed(fwds),
                                info=fixed(info),
                                axes=fixed(axes),
                                fig_axes=fixed(fig_axes))

output = interactive_plot.children[-1]
output.layout.height = '600px'
interactive_plot

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, continuous_update=False, description='azimuth φ (deg)', max=360, styl…